# The Battle of Neighborhoods

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize

!pip install geocoder
!pip install geopy

from geopy.geocoders import Nominatim
import requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium

import folium

from bs4 import BeautifulSoup 
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
!pip install folium
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL)
print("Done")

Done


In [3]:
soup = BeautifulSoup(r.content, 'html5lib') 
table_post = soup.find('table')
fields = table_post.find_all('td')

postcode = []
borough = []
neighbourhood = []

for i in range(0, len(fields), 3):
    postcode.append(fields[i].text.strip())
    borough.append(fields[i+1].text.strip())
    neighbourhood.append(fields[i+2].text.strip())
        
df_pc = pd.DataFrame(data=[postcode, borough, neighbourhood]).transpose()
df_pc.columns = ['Postcode', 'Borough', 'Neighborhood']
df_pc.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df_pc['Borough'].replace('Not assigned', np.nan, inplace=True)
df_pc.dropna(subset=['Borough'], inplace=True)

df_pc.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
df_pc['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

In [6]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns = ['Postcode', 'Latitude', 'Longitude']
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df_pos = pd.merge(df_pc, df_geo, on=['Postcode'], how='inner')
canada_data = df_pos
canada_data.drop(['Postcode'], axis=1,inplace=True)
canada_data

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
98,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                   
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=150'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
CLIENT_ID = 'VKHLA4CXE2KT3HBDCEORA1TAKGRZT3S4QULA5VSQPC30CXRB' 
CLIENT_SECRET = '30G5ZY2W4VHXLW5IVK02BSHE3ZVNH3ROA1ILL1TGHG3G402H'
VERSION = '20180605'

In [10]:
venues = getNearbyVenues(names=canada_data['Neighborhood'],latitudes=canada_data['Latitude'],longitudes=canada_data['Longitude'])

In [11]:
venues.shape

(8642, 7)

In [12]:
venues.to_csv('venues around canada neighborhoods.csv')

In [13]:
venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
4,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
5,Parkwoods,43.753259,-79.329656,LCBO,43.757774,-79.314257,Liquor Store
6,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
7,Parkwoods,43.753259,-79.329656,The Captain's Boil,43.754986,-79.349524,Seafood Restaurant
8,Parkwoods,43.753259,-79.329656,Me Va Me Kitchen Express,43.754957,-79.351894,Mediterranean Restaurant
9,Parkwoods,43.753259,-79.329656,Menchie's,43.754764,-79.350013,Frozen Yogurt Shop


In [32]:
dummies_df = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
dummies_df['Neighborhoods'] = venues['Neighborhood']
dummies_df.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,...,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit,Neighborhoods
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods


In [33]:
column_name= "Neighborhoods"
first_column = dummies_df.pop(column_name)

In [35]:
dummies_df.insert(0,column_name,first_column)

In [36]:
dummies_df.head()

,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
grouped = dummies_df.groupby(["Neighborhoods"]).mean().reset_index()
grouped.head()

,Neighborhoods,Accessories Store,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.018519,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.019231,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0


In [40]:
gym = grouped[['Neighborhoods','Gym']]
gym.head()

,Neighborhoods,Gym
0,Agincourt,0.010000
1,"Alderwood, Long Branch",0.010000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.018519
3,Bayview Village,0.000000
4,"Bedford Park, Lawrence Manor East",0.010000


In [41]:
from sklearn.cluster import KMeans
toclusters = 3

gym_clustering = gym.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=1)
kmeans.fit_transform(gym_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]



array([0, 0, 1, 2, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 2, 2, 2, 2, 1, 0])

In [49]:
gym_clustered = gym.copy()
gym_clustered['Cluster']=kmeans.labels_
gym_clustered.columns = ['Neighborhood','Gym','Cluster']
gym_clustered.head()

,Neighborhood,Gym,Cluster
0,Agincourt,0.010000,0
1,"Alderwood, Long Branch",0.010000,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.018519,1
3,Bayview Village,0.000000,2
4,"Bedford Park, Lawrence Manor East",0.010000,0


In [53]:
gym_clustered_merged = pd.merge(gym_clustered,canada_data, on='Neighborhood')
gym_clustered_merged.sort_values(by='Gym',ascending=False, inplace=True)
gym_clustered_merged.head()

,Neighborhood,Gym,Cluster,Borough,Latitude,Longitude
91,Victoria Village,0.060000,1,North York,43.725882,-79.315572
12,Cedarbrae,0.040000,1,Scarborough,43.773136,-79.239476
31,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.030303,1,Etobicoke,43.643515,-79.577201
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.030000,1,Downtown Toronto,43.628947,-79.394420
39,"Harbourfront East, Union Station, Toronto Islands",0.030000,1,Downtown Toronto,43.640816,-79.381752


In [55]:
!pip install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent= "canada_explorer")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Toronto is %f and %f" %(latitude,longitude))

The latitude and longitude of Toronto is 43.653482 and -79.383935


In [58]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i+x+(i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gym_clustered_merged['Latitude'], gym_clustered_merged['Longitude'], gym_clustered_merged['Neighborhood'], gym_clustered_merged['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
gym_clustered_merged.loc[gym_clustered_merged['Cluster'] == 0]

,Neighborhood,Gym,Cluster,Borough,Latitude,Longitude
22,Don Mills,0.015789,0,North York,43.745906,-79.352188
23,Don Mills,0.015789,0,North York,43.725900,-79.340923
94,Weston,0.015625,0,York,43.706876,-79.518188
99,Woburn,0.015152,0,Scarborough,43.770992,-79.216917
49,"Kingsview Village, St. Phillips, Martin Grove ...",0.014706,0,Etobicoke,43.688905,-79.554724
74,"Runnymede, The Junction North",0.013158,0,York,43.673185,-79.487262
66,Parkwoods,0.010870,0,North York,43.753259,-79.329656
24,"Dorset Park, Wexford Heights, Scarborough Town...",0.010526,0,Scarborough,43.757410,-79.273304
37,"Golden Mile, Clairlea, Oakridge",0.010526,0,Scarborough,43.711112,-79.284577
96,Willowdale,0.010204,0,North York,43.770120,-79.408493


In [62]:
gym_clustered_merged.loc[gym_clustered_merged['Cluster'] == 1]

,Neighborhood,Gym,Cluster,Borough,Latitude,Longitude
91,Victoria Village,0.060000,1,North York,43.725882,-79.315572
12,Cedarbrae,0.040000,1,Scarborough,43.773136,-79.239476
31,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.030303,1,Etobicoke,43.643515,-79.577201
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.030000,1,Downtown Toronto,43.628947,-79.394420
39,"Harbourfront East, Union Station, Toronto Islands",0.030000,1,Downtown Toronto,43.640816,-79.381752
75,Scarborough Village,0.027027,1,Scarborough,43.744734,-79.239476
72,"Rouge Hill, Port Union, Highland Creek",0.025641,1,Scarborough,43.784535,-79.160497
41,Hillcrest Village,0.025316,1,North York,43.803762,-79.363452
62,"Northwood Park, York University",0.023529,1,North York,43.767980,-79.487262
52,Leaside,0.022989,1,East York,43.709060,-79.363452


In [61]:
gym_clustered_merged.loc[gym_clustered_merged['Cluster'] == 2]

,Neighborhood,Gym,Cluster,Borough,Latitude,Longitude
21,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.0,2,York,43.691116,-79.476013
85,"The Danforth West, Riverdale",0.0,2,East Toronto,43.679557,-79.352188
83,"The Annex, North Midtown, Yorkville",0.0,2,Central Toronto,43.672710,-79.405678
25,Downsview,0.0,2,North York,43.737473,-79.464763
17,"Cliffside, Cliffcrest, Scarborough Village West",0.0,2,Scarborough,43.716316,-79.239476
16,"Clarks Corners, Tam O'Shanter, Sullivan",0.0,2,Scarborough,43.781638,-79.304302
89,"University of Toronto, Harbord",0.0,2,Downtown Toronto,43.662696,-79.400049
90,Upper Rouge,0.0,2,Scarborough,43.836125,-79.205636
27,Downsview,0.0,2,North York,43.728496,-79.495697
15,Church and Wellesley,0.0,2,Downtown Toronto,43.665860,-79.383160


In [63]:
veg_res = grouped[['Neighborhoods','Vegetarian / Vegan Restaurant']]

In [64]:
from sklearn.cluster import KMeans
toclusters = 3

veg_res_clustering = veg_res.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans_veg_res = KMeans(n_clusters=toclusters, random_state=1)
kmeans_veg_res.fit_transform(veg_res_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_veg_res.labels_[0:20]


array([0, 0, 0, 0, 0, 2, 0, 1, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2])

In [67]:
veg_res_clustered = veg_res.copy()
veg_res_clustered['Cluster']=kmeans_veg_res.labels_
veg_res_clustered.columns = ['Neighborhood','Veg / Vegan Restaurant','Cluster']
veg_res_clustered.head()

,Neighborhood,Veg / Vegan Restaurant,Cluster
0,Agincourt,0.0,0
1,"Alderwood, Long Branch",0.0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0
3,Bayview Village,0.0,0
4,"Bedford Park, Lawrence Manor East",0.0,0


In [68]:
veg_res_clustered_merged = pd.merge(veg_res_clustered,canada_data, on='Neighborhood')
veg_res_clustered_merged.sort_values(by='Veg / Vegan Restaurant',ascending=False, inplace=True)
veg_res_clustered_merged.head()

,Neighborhood,Veg / Vegan Restaurant,Cluster,Borough,Latitude,Longitude
14,Christie,0.04,1,Downtown Toronto,43.669542,-79.422564
53,"Little Portugal, Trinity",0.03,1,West Toronto,43.647927,-79.419750
48,"Kensington Market, Chinatown, Grange Park",0.03,1,Downtown Toronto,43.653206,-79.400049
39,"Harbourfront East, Union Station, Toronto Islands",0.02,1,Downtown Toronto,43.640816,-79.381752
18,"Commerce Court, Victoria Hotel",0.02,1,Downtown Toronto,43.648198,-79.379817


In [55]:
!pip install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent= "canada_explorer")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Toronto is %f and %f" %(latitude,longitude))

The latitude and longitude of Toronto is 43.653482 and -79.383935


In [69]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(3)
ys = [i+x+(i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(veg_res_clustered_merged['Latitude'], veg_res_clustered_merged['Longitude'], veg_res_clustered_merged['Neighborhood'], veg_res_clustered_merged['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [70]:
veg_res_clustered_merged.loc[veg_res_clustered_merged['Cluster'] == 0]

,Neighborhood,Veg / Vegan Restaurant,Cluster,Borough,Latitude,Longitude
63,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.0,0,Etobicoke,43.636258,-79.498509
74,"Runnymede, The Junction North",0.0,0,York,43.673185,-79.487262
62,"Northwood Park, York University",0.0,0,North York,43.767980,-79.487262
72,"Rouge Hill, Port Union, Highland Creek",0.0,0,Scarborough,43.784535,-79.160497
64,"Parkdale, Roncesvalles",0.0,0,West Toronto,43.648960,-79.456325
...,...,...,...,...,...,...
38,"Guildwood, Morningside, West Hill",0.0,0,Scarborough,43.763573,-79.188711
36,Glencairn,0.0,0,North York,43.709577,-79.445073
32,"Fairview, Henry Farm, Oriole",0.0,0,North York,43.778517,-79.346556
31,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.0,0,Etobicoke,43.643515,-79.577201


In [71]:
veg_res_clustered_merged.loc[veg_res_clustered_merged['Cluster'] == 1]

,Neighborhood,Veg / Vegan Restaurant,Cluster,Borough,Latitude,Longitude
14,Christie,0.040000,1,Downtown Toronto,43.669542,-79.422564
53,"Little Portugal, Trinity",0.030000,1,West Toronto,43.647927,-79.419750
48,"Kensington Market, Chinatown, Grange Park",0.030000,1,Downtown Toronto,43.653206,-79.400049
39,"Harbourfront East, Union Station, Toronto Islands",0.020000,1,Downtown Toronto,43.640816,-79.381752
18,"Commerce Court, Victoria Hotel",0.020000,1,Downtown Toronto,43.648198,-79.379817
83,"The Annex, North Midtown, Yorkville",0.020000,1,Central Toronto,43.672710,-79.405678
88,"Toronto Dominion Centre, Design Exchange",0.020000,1,Downtown Toronto,43.647177,-79.381576
33,"First Canadian Place, Underground city",0.020000,1,Downtown Toronto,43.648429,-79.382280
69,"Richmond, Adelaide, King",0.020000,1,Downtown Toronto,43.650571,-79.384568
7,"Brockton, Parkdale Village, Exhibition Place",0.020000,1,West Toronto,43.636847,-79.428191


In [72]:
veg_res_clustered_merged.loc[veg_res_clustered_merged['Cluster'] == 2]

,Neighborhood,Veg / Vegan Restaurant,Cluster,Borough,Latitude,Longitude
37,"Golden Mile, Clairlea, Oakridge",0.010526,2,Scarborough,43.711112,-79.284577
34,Forest Hill North & West,0.010000,2,Central Toronto,43.696948,-79.411307
60,North Toronto West,0.010000,2,Central Toronto,43.715383,-79.405678
82,"Summerhill West, Rathnelly, South Hill, Forest...",0.010000,2,Central Toronto,43.686412,-79.400049
35,"Garden District, Ryerson",0.010000,2,Downtown Toronto,43.657162,-79.378937
77,St. James Town,0.010000,2,Downtown Toronto,43.651494,-79.375418
84,The Beaches,0.010000,2,East Toronto,43.676357,-79.293031
71,Roselawn,0.010000,2,Central Toronto,43.711695,-79.416936
70,Rosedale,0.010000,2,Downtown Toronto,43.679563,-79.377529
80,Stn A PO Boxes,0.010000,2,Downtown Toronto,43.646435,-79.374846


In [74]:
gym_vegan = grouped[['Neighborhoods','Vegetarian / Vegan Restaurant','Gym']]

,Neighborhoods,Vegetarian / Vegan Restaurant,Gym
0,Agincourt,0.0,0.01
1,"Alderwood, Long Branch",0.0,0.01


In [90]:
from sklearn.cluster import KMeans
toclusters = 4

gym_vegan_clustering = gym_vegan.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans_gym_vegan = KMeans(n_clusters=toclusters, random_state=1)
kmeans_gym_vegan.fit_transform(gym_vegan_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_gym_vegan.labels_[0:20]


array([0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 2, 3, 0, 1, 2, 2, 2, 1, 0])

In [91]:
gym_vegan_clustered = gym_vegan.copy()
gym_vegan_clustered['Cluster']=kmeans_gym_vegan.labels_
gym_vegan_clustered.columns = ['Neighborhood','Veg / Vegan Restaurant','Gym','Cluster']
gym_vegan_clustered.head()

,Neighborhood,Veg / Vegan Restaurant,Gym,Cluster
0,Agincourt,0.0,0.010000,0
1,"Alderwood, Long Branch",0.0,0.010000,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.018519,0
3,Bayview Village,0.0,0.000000,2
4,"Bedford Park, Lawrence Manor East",0.0,0.010000,0


In [92]:
gym_vegan_clustered_merged = pd.merge(gym_vegan_clustered,canada_data, on='Neighborhood')
gym_vegan_clustered_merged.head()

,Neighborhood,Veg / Vegan Restaurant,Gym,Cluster,Borough,Latitude,Longitude
0,Agincourt,0.0,0.010000,0,Scarborough,43.794200,-79.262029
1,"Alderwood, Long Branch",0.0,0.010000,0,Etobicoke,43.602414,-79.543484
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.018519,0,North York,43.754328,-79.442259
3,Bayview Village,0.0,0.000000,2,North York,43.786947,-79.385975
4,"Bedford Park, Lawrence Manor East",0.0,0.010000,0,North York,43.733283,-79.419750


In [93]:
!pip install geopy
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent= "canada_explorer")
location = geolocator.geocode('Toronto')
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude of Toronto is %f and %f" %(latitude,longitude))

The latitude and longitude of Toronto is 43.653482 and -79.383935


In [96]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(4)
ys = [i+x+(i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(gym_vegan_clustered_merged['Latitude'], gym_vegan_clustered_merged['Longitude'], gym_vegan_clustered_merged['Neighborhood'], gym_vegan_clustered_merged['Cluster']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [101]:
gym_vegan_cluster0 = gym_vegan_clustered_merged.loc[gym_vegan_clustered_merged['Cluster'] == 0]

In [102]:
gym_vegan_cluster1 = gym_vegan_clustered_merged.loc[gym_vegan_clustered_merged['Cluster'] == 1]

In [103]:
gym_vegan_cluster2 = gym_vegan_clustered_merged.loc[gym_vegan_clustered_merged['Cluster'] == 2]

In [105]:
gym_vegan_cluster3 = gym_vegan_clustered_merged.loc[gym_vegan_clustered_merged['Cluster'] == 3]

In [109]:
gym_vegan_cluster0[['Veg / Vegan Restaurant','Gym']].mean()

Veg / Vegan Restaurant    0.002788
Gym                       0.015283
dtype: float64

In [110]:
gym_vegan_cluster1[['Veg / Vegan Restaurant','Gym']].mean()

Veg / Vegan Restaurant    0.023103
Gym                       0.012500
dtype: float64

In [111]:
gym_vegan_cluster2[['Veg / Vegan Restaurant','Gym']].mean()

Veg / Vegan Restaurant    0.000857
Gym                       0.000000
dtype: float64

In [112]:
gym_vegan_cluster3[['Veg / Vegan Restaurant','Gym']].mean()

Veg / Vegan Restaurant    0.005
Gym                       0.050
dtype: float64